In [1]:
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from alexnet import AlexNet
from sklearn.utils import shuffle

In [2]:
# TODO: Load traffic signs data.
# training_file = "./data/train.p"
# validation_file="./data/valid.p"
# testing_file = "./data/test.p"

# with open(training_file, mode='rb') as f:
#     train = pickle.load(f)
# with open(validation_file, mode='rb') as f:
#     valid = pickle.load(f)
# with open(testing_file, mode='rb') as f:
#     test = pickle.load(f)

with open('./train.p', mode='rb') as f:
    data = pickle.load(f)

In [3]:
# TODO: Split data into training and validation sets.
# X_train, y_train = train['features'], train['labels']
# X_valid, y_valid = valid['features'], valid['labels']
# X_test, y_test = test['features'], test['labels']

X_train, X_valid, y_train, y_valid = train_test_split(data['features'], data['labels'], 
                                                  test_size=0.20, random_state=0)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(31367, 32, 32, 3) (7842, 32, 32, 3) (31367,) (7842,)


In [4]:
nb_classes = 43
model_path = "/tmp/alex.ckpt"

EPOCHS = 10
BATCH_SIZE = 128
LEARNING_RATE = 0.001

In [5]:
# TODO: Define placeholders and resize operation.
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
resized = tf.image.resize_images(x, (227, 227))

#y = tf.placeholder(tf.int32, None)
y = tf.placeholder(tf.int64, None)
# one_hot_y = tf.one_hot(y, nb_classes)

In [6]:
# TODO: pass placeholder as first argument to `AlexNet`.
fc7 = AlexNet(resized, feature_extract=True)
# NOTE: `tf.stop_gradient` prevents the gradient from flowing backwards
# past this point, keeping the weights before and up to `fc7` frozen.
# This also makes training faster, less work to do!
fc7 = tf.stop_gradient(fc7)

In [7]:
# TODO: Add the final layer for traffic sign classification.
# HINT: Look at the final layer definition in alexnet.py to get an idea of what this
# should look like.
shape = (fc7.get_shape().as_list()[-1], nb_classes)  # use this shape for the weight matrix
fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.nn.xw_plus_b(fc7, fc8W, fc8b)
logits.shape
# probs = tf.nn.softmax(logits)

TensorShape([Dimension(None), Dimension(43)])

In [8]:
# TODO: Define loss, training, accuracy operations.
# HINT: Look back at your traffic signs project solution, you may
# be able to reuse some the code.

# cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE)
training_operation = optimizer.minimize(loss_operation, var_list=[fc8W, fc8b])

In [10]:
# correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
correct_prediction = tf.equal(tf.argmax(logits, 1), y)
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [12]:
# TODO: Train and evaluate the feature extraction model.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print("Epoch, Validation Accuracy")
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        print((i+1), validation_accuracy)
        
    saver.save(sess, model_path)
    print("Model saved")

Training...
Epoch, Validation Accuracy
1 0.872864065487
2 0.923106350314
3 0.939811272528
4 0.948482529982
5 0.959449120138
6 0.961489415981
7 0.957791379765
8 0.966080081627
9 0.972073450787
10 0.971435858215
Model saved


In [15]:
with tf.Session() as sess:
    saver.restore(sess, model_path)

    valid_accuracy = evaluate(X_valid, y_valid)
    print("Valid Accuracy = {:.3f}".format(valid_accuracy))

INFO:tensorflow:Restoring parameters from /tmp/alex.ckpt
Valid Accuracy = 0.971
